In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import datetime
from datetime import timedelta as delta
from matplotlib import rc
from mpl_toolkits.basemap import Basemap
%pylab inline

In [ ]:
class ParticleData(object):
    def __init__(self):
        self.id = []
        self.ends = []

    def trajends(self):
        lonends = [self.lon[i,self.ends[i]] for i in range(self.lon.shape[0])]
        latends = [self.lat[i,self.ends[i]] for i in range(self.lat.shape[0])]
        return lonends, latends

def load_particles_file(fname, varnames):
    T = ParticleData()
    pfile = Dataset(fname, 'r')
    T.id = pfile.variables['trajectory'][:]

    if 'trajectory' in pfile.dimensions:  # file in array format
        for v in varnames:
            if v is 'startloc':
                setattr(T, v, pfile.variables[v][:,0])
            else:
                setattr(T, v, pfile.variables[v][:])
        nid = T.lon.shape[0]
        nobs = T.lon.shape[1]
        T.ends = np.empty((nid, 1), dtype=np.int)
        for i in range(nid):
            I = np.nonzero(T.age[i,:])[0]
            I = nobs+1 if I.size==0 else I[0]
            T.lon[i,0:I-1] = np.nan
            T.lat[i,0:I-1] = np.nan            
            if np.isnan(T.lon[i,:]).any():
                T.ends[i] = np.where(np.isnan(T.lon[i,:]))[0][0]-1
            else:
                T.ends[i] = T.lon.shape[-1]-1
    T.time_origin = datetime.datetime(1900, 12, 31)
    return T

In [ ]:
def load_exps(startdate):
    Ts = {}
    for s, marker, name, color in zip([38, 40], ['s','o'], ['EAC', 'Coastal'], ['C0', 'C1']):
        fname = 'microbes_eac_hycom_particles_starts'+startdate.strftime('%Y%m%d')+'_'+str(s)+'.nc'
        Ts[s] = load_particles_file(fname, ['lon', 'lat', 'temp', 'age', 'time'])
        Ts[s].time = np.ma.filled(np.mean(Ts[s].time,0), np.nan)
        Ts[s].time = [Ts[s].time_origin + datetime.timedelta(seconds=Ts[s].time[t]) for t in range(Ts[s].time.size)]
        Ts[s].marker = marker
        Ts[s].name = name
        Ts[s].color = color
        Ts[s].startdate = startdate
    return Ts

In [ ]:
## Main Figure for manuscript
pylab.rcParams['figure.figsize'] = (14, 14)
fig = plt.figure()
t0 = datetime.date(2015, 6, 13)  # True sampling start date

Ts = load_exps(startdate=t0)
minlon = min([np.amin(Ts[z].lon) for z in Ts])
maxlon = max([np.amax(Ts[z].lon) for z in Ts])
minlat = min([np.amin(Ts[z].lat) for z in Ts])
maxlat = max([np.amax(Ts[z].lat) for z in Ts])
mintemp = min([np.amin(Ts[z].temp) for z in Ts])
maxtemp = max([np.amax(Ts[z].temp) for z in Ts])

for i, z in enumerate(Ts):
    ax = fig.add_axes([0.125+i*0.37, 0.45, 0.35, 0.4])
    ttl = ax.set_title('Site '+str(i+1))
    m = Basemap(projection='cea', ax=ax, llcrnrlat=minlat-1, urcrnrlat=maxlat+1,\
                llcrnrlon=minlon-10, urcrnrlon=maxlon+1, resolution='l')
    m.drawcoastlines()
    m.fillcontinents(color='burlywood')
    xs, ys = m(Ts[z].lon[:].flatten(), Ts[z].lat[:,:].flatten())
    scat = m.scatter(xs, ys, c=Ts[z].temp.flatten(), s=10, edgecolor='k')
    m.scatter(xs[0], ys[0], c='r', s=100, edgecolor='k')
    scat.set_clim([mintemp, maxtemp])
cax = fig.add_axes([0.86, 0.485, 0.02, 0.33])
cb = plt.colorbar(scat, orientation='vertical', cax=cax)
cb.ax.set_title('[$^{\circ}$C]')

ax = fig.add_subplot(2, 2, (3,4))
for i, z in enumerate(Ts):
    mn = np.nanmean(Ts[z].temp, 0)
    st = np.nanstd(Ts[z].temp, 0)
    xax = np.arange(0, len(mn))
    ax.plot(xax, mn, label='Site '+str(i+1), color=Ts[z].color)
    ax.fill_between(xax, mn-st, mn+st, alpha=0.25, facecolor=Ts[z].color)
    
ax.invert_xaxis()
ax.set_ylim(19, 28)
ax.set_xlim(85, 0)
ax.grid('on')
ax.set_ylabel('Average temperature $\pm$ one standard deviation [$^{\circ}$C]')
ax.set_xlabel('Days before sampling')
ax.legend()
plt.show()

In [ ]:
## Print data for statistical analysis

for i, z in enumerate(Ts):
    mn[z] = np.nanmean(Ts[z].temp, 0)
    st[z] = np.nanstd(Ts[z].temp, 0)

print 'day', 'mean_site1', 'std_site1', 'mean_site2', 'std_site2'
for i in range(len(mn[z])):
    print i, mn[38][i], st[38][i], mn[40][i], st[40][i]

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)
def makemov(Ts):
    fig = plt.figure()

    frames = np.arange(0, len(Ts[38].time))
    
    minlon = min([np.amin(Ts[z].lon) for z in Ts])
    maxlon = max([np.amax(Ts[z].lon) for z in Ts])
    minlat = min([np.amin(Ts[z].lat) for z in Ts])
    maxlat = max([np.amax(Ts[z].lat) for z in Ts])
    mintemp = min([np.amin(Ts[z].temp) for z in Ts])
    maxtemp = max([np.amax(Ts[z].temp) for z in Ts])
    
    
    def setmap(i, T):
        ax = fig.add_subplot(1, len(Ts), i+1)
        ttl = ax.set_title(Ts[z].name + ' particles on ' + T.time[frames[0]].strftime("%Y-%m-%d"))
        m = Basemap(projection='cea', ax=ax, llcrnrlat=minlat-1, urcrnrlat=maxlat+1,\
                    llcrnrlon=minlon-1, urcrnrlon=maxlon+1, resolution='i')
        m.drawcoastlines()
        m.fillcontinents(color='burlywood')
        xs, ys = m(T.lon[:, frames[0]], T.lat[:, frames[0]])
        m.scatter(xs, ys, c='r', s=100, edgecolor='k')
        return m, ttl
    
    def setscat(T, m):
        xs, ys = m(T.lon[:, frames[0]], T.lat[:, frames[0]])
        scat = m.scatter(xs, ys, c=T.temp[:, frames[0]], s=45, edgecolor='k', marker=Ts[z].marker)
        scat.set_clim([mintemp, maxtemp])
        plt.colorbar(scat, orientation='horizontal')
        return scat

    m = {}
    scat = {}
    ttl = {}
    for i, z in enumerate(Ts):
        m[z], ttl[z] = setmap(i, Ts[z])
        scat[z] = setscat(Ts[z], m[z])
    
    def animate(t):
        for z in Ts:
            scat[z].set_offsets(np.matrix(m[z](Ts[z].lon[:, t], Ts[z].lat[:, t])).transpose())
            scat[z].set_array(Ts[z].temp[:, t])
            ttl[z].set_text(Ts[z].name + ' particles on ' + Ts[z].time[t].strftime("%Y-%m-%d"))
        return scat

    rc('animation', html='html5')
    anim = animation.FuncAnimation(fig, animate, frames=frames,
                                   interval=100, blit=False)
    plt.close()
    return anim

Ts = load_exps(startdate=datetime.date(2015, 6, 13)-delta(days=28))
makemov(Ts)

In [ ]:
for z in Ts:
    mn = np.nanmean(Ts[z].temp, 0)
    st = np.nanstd(Ts[z].temp, 0)
    plt.plot(Ts[z].time, mn, label=Ts[z].name, color=Ts[z].color)
    plt.fill_between(Ts[z].time, mn-st, mn+st, alpha=0.25, facecolor=Ts[z].color)

plt.ylabel('Average temperature [$^{\circ}$C]')
plt.xlabel('Date')
plt.title('Mean +/- 1 std longitude as a function of time')
plt.legend()
plt.grid()
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 14)
fig = plt.figure()
t0 = datetime.date(2015, 6, 13)  # True sampling start date

for i, t in enumerate(np.arange(-4, 5, 1)*7):
    startdate = delta(days=t) + t0
    Ts = load_exps(startdate=startdate)
    ax = fig.add_subplot(3, 3, i+1)
    for z in Ts:
        mn = np.nanmean(Ts[z].temp, 0)
        st = np.nanstd(Ts[z].temp, 0)
        xax = np.arange(0, len(mn))
        ax.plot(xax, mn, label=Ts[z].name, color=Ts[z].color)
        ax.fill_between(xax, mn-st, mn+st, alpha=0.25, facecolor=Ts[z].color)
    ax.invert_xaxis()
    ax.set_ylim(19, 28)
    ax.set_xlim(86, 0)
    ax.grid('on')
    if t == 0:
        [i.set_linewidth(2) for i in ax.spines.itervalues()]
        ax.set_title('Sampling on '+Ts[z].startdate.strftime("%Y-%m-%d"), fontweight='bold')
        ax.legend()
    else:
        ax.set_title('Sampling on '+Ts[z].startdate.strftime("%Y-%m-%d"))

fig.suptitle('Temperature histories from Coastal and EAC sampling sites', fontsize=14)
fig.text(0.08, 0.5, 'Average temperature $\pm$ one standard deviation [$^{\circ}$C]', va='center', rotation='vertical', fontsize=14)
fig.text(0.5, 0.05, 'Days before sampling', va='center', fontsize=14)
subplots_adjust(top=0.94)
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 14)
fig = plt.figure()
t0 = datetime.date(2015, 6, 13)  # True sampling start date

for i, t in enumerate(np.arange(-4, 5, 1)*7):
    startdate = delta(days=t) + t0
    Ts = load_exps(startdate=startdate)
    ax = fig.add_subplot(3, 3, i+1)
    for z in Ts:
        mn = -np.nanmean(Ts[z].lat, 0)
        st = -np.nanstd(Ts[z].lat, 0)
        xax = np.arange(0, len(mn))
        ax.plot(xax, mn, label=Ts[z].name, color=Ts[z].color)
        ax.fill_between(xax, mn-st, mn+st, alpha=0.25, facecolor=Ts[z].color)
    ax.invert_xaxis()
    ax.invert_yaxis()
    ax.set_ylim(38, 20)
    ax.set_xlim(86, 0)
    ax.grid('on')
    if t == 0:
        [i.set_linewidth(2) for i in ax.spines.itervalues()]
        ax.set_title('Sampling on '+Ts[z].startdate.strftime("%Y-%m-%d"), fontweight='bold')
        ax.legend()
    else:
        ax.set_title('Sampling on '+Ts[z].startdate.strftime("%Y-%m-%d"))

fig.suptitle('Latitude histories from Coastal and EAC sampling sites', fontsize=14)
fig.text(0.08, 0.5, 'Average latitude $\pm$ one standard deviation [$^{\circ}$S]', va='center', rotation='vertical', fontsize=14)
fig.text(0.5, 0.05, 'Days before sampling', va='center', fontsize=14)
subplots_adjust(top=0.94)
plt.show()